<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bf684ab0950a8f2cb224012fb5e3468c40b768d4cf2147f234fc8ba1ec8421f6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-21 10:12:52
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: -1.02 L (-0.71%)
Current PnL: -22.22 L (-14.56%)
CY Booked + Current PnL: -8.33 L (-5.46%)
-------------------
Total profit:  1.60 L
Total loss:  -23.82 L
-------------------
Total Booked + Current PnL: 18.65 L (14.85%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.67%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.63 L (61.04%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.11,-14.13,16.56,0.09,14331.0,-14238.0,86539.0,100.85,50.0,M-SC,4.15,253.0,-0.99,0.60,13.69,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.32,-8.39,9.18,0.02,14680.0,-14649.0,159910.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.09,OX40N,NTT,PAINTS
83,VOLTAS,1530.00,-0.46,9.94,8.87,19.69,18698.0,19053.0,210795.0,-0.80,56.0,X-MC,3.59,78.0,1.02,1.47,17.41,XY25,NTT,AC
35,ICICIGI,2252.93,-1.28,7.31,12.00,20.19,22448.0,12750.0,187070.0,-15.78,52.0,X-MC,7.68,68.0,0.57,1.30,23.53,X40,ATH,INSURANCE
43,ITC,452.00,0.04,-0.90,11.44,10.44,22690.0,-1800.0,198338.0,-40.74,45.0,X-LC,0.97,5.0,-0.08,1.38,4.61,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIS,528.0,2.69,-23.80,59.13,21.26,50029.0,-26423.0,84609.0,1996.37,46.0,H-SC,3.23,166.0,-0.53,0.59,14.32,OX40N,NTT,MISC
40,INDUSINDBK,1800.0,1.52,-37.84,113.78,32.89,54608.0,-29212.0,47994.0,-693.00,63.0,L-MC,5.36,259.0,-0.53,0.33,32.17,XR,NTT,BANKS
52,MEDANTA,1486.0,0.70,2.00,21.64,24.08,28022.0,2540.0,129490.0,-6.29,42.0,X-SC,4.52,89.0,0.09,0.90,22.01,XY24,NTT,HEALTHCARE
76,TMPV,600.0,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.68,XY24,NTT,AUTO
12,BERGEPAINT,680.0,0.42,5.57,14.75,21.14,35400.0,12664.0,240003.0,-1.84,75.0,X-MC,3.56,74.0,0.36,1.67,35.28,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-3.27,-15.28,81.17,53.50,207514.0,-46097.0,255654.0,-59.04,46.0,X-MC,3.82,58.0,-0.22,1.78,13.55,XY24,NTT,FMCG
38,INDIAMART,4810.62,-2.73,-0.03,106.77,106.72,131652.0,-32.0,123304.0,-50.44,34.0,H-SC,4.38,139.0,-0.00,0.86,23.35,AR,ATH,MISC
56,QUESS,424.00,-2.10,-28.83,100.71,42.84,46592.0,-18742.0,46264.0,-53.17,37.0,X-SC,37.14,83.0,-0.40,0.32,0.24,XY24,NTT,MISC
48,KPIGREEN,730.66,-2.09,-8.55,60.69,46.95,69541.0,-10713.0,114584.0,-15.48,32.0,H-SC,7.43,154.0,-0.15,0.80,43.55,MH,ATH,POWER
69,STARHEALTH,761.00,-1.82,-3.95,53.32,47.26,143971.0,-11109.0,270014.0,31.02,52.0,H-SC,15.66,171.0,-0.08,1.88,44.67,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-2.73,-0.03,106.77,106.72,131652.0,-32.0,123304.0,-50.44,34.0,H-SC,4.38,139.0,-0.0,0.86,23.35,AR,ATH,MISC
85,WIPRO,420.00,-0.82,0.18,71.95,72.27,122138.0,306.0,169754.0,-14.25,52.0,M-LC,5.06,99.0,0.0,1.18,6.37,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.05,-3.82,116.16,107.91,164956.0,-5636.0,142008.0,-21.77,36.0,M-SC,10.20,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.32,-8.39,9.18,0.02,14680.0,-14649.0,159910.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.09,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.11,-14.13,16.56,0.09,14331.0,-14238.0,86539.0,100.85,50.0,M-SC,4.15,253.0,-0.99,0.60,13.69,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.36,-21.06,43.74,13.47,93112.0,-56790.0,212877.0,-67.53,31.0,H-SC,1.67,158.0,-0.61,1.48,8.24,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.05,-23.20,30.19,-0.01,29634.0,-29648.0,98157.0,19986.96,64.0,M-SC,1.54,233.0,-1.00,0.68,11.15,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.82,0.18,71.95,72.27,122138.0,306.0,169754.0,-14.25,52.0,M-LC,5.06,99.0,0.0,1.18,6.37,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-2.73,-0.03,106.77,106.72,131652.0,-32.0,123304.0,-50.44,34.0,H-SC,4.38,139.0,-0.00,0.86,23.35,AR,ATH,MISC
85,WIPRO,420.00,-0.82,0.18,71.95,72.27,122138.0,306.0,169754.0,-14.25,52.0,M-LC,5.06,99.0,0.00,1.18,6.37,XR,NTT,IT
25,FINCABLES,1641.55,-1.05,-3.82,116.16,107.91,164956.0,-5636.0,142008.0,-21.77,36.0,M-SC,10.20,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
37,IEX,219.00,0.21,-2.66,52.69,48.63,102553.0,-5319.0,194635.0,-35.29,59.0,H-SC,15.34,136.0,-0.05,1.36,8.59,XR,NTT,MISC
84,WHIRLPOOL,2270.00,-0.85,-5.61,92.73,81.92,100480.0,-6440.0,108358.0,-44.68,31.0,M-SC,6.17,236.0,-0.06,0.75,28.28,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.50,-35.21,108.68,35.20,90601.0,-45305.0,83365.0,9.31,23.0,X-SC,12.00,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.68,XY24,NTT,AUTO
34,HONAUT,58357.33,-0.49,-15.23,69.08,43.32,95372.0,-24812.0,138060.0,-27.20,25.0,X-SC,1.68,90.0,-0.26,0.96,6.00,X40N,ATH,ELECTRICAL
46,JSWINFRA,342.00,-0.72,-7.60,24.70,15.23,45724.0,-15228.0,185119.0,-21.02,28.0,X-MC,6.34,66.0,-0.33,1.29,22.20,X40N,NTT,REALTY
9,BAJAJHFL,181.50,-0.71,-13.58,71.78,48.45,124382.0,-27224.0,173282.0,-19.87,30.0,X-MC,7.29,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.68,XY24,NTT,AUTO
43,ITC,452.00,0.04,-0.90,11.44,10.44,22690.0,-1800.0,198338.0,-40.74,45.0,X-LC,0.97,5.0,-0.08,1.38,4.61,X40,NTT,FMCG
36,ICICIPRULI,790.00,-0.73,1.77,28.69,30.97,51606.0,3135.0,179873.0,-22.18,51.0,X-MC,1.51,75.0,0.06,1.25,14.56,X40,ATH,INSURANCE
34,HONAUT,58357.33,-0.49,-15.23,69.08,43.32,95372.0,-24812.0,138060.0,-27.20,25.0,X-SC,1.68,90.0,-0.26,0.96,6.00,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.52,-22.60,112.09,64.15,206436.0,-53781.0,184170.0,-54.57,48.0,X-SC,1.82,82.0,-0.26,1.28,3.31,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.50,-35.21,108.68,35.20,90601.0,-45305.0,83365.0,9.31,23.0,X-SC,12.00,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.71,-13.58,71.78,48.45,124382.0,-27224.0,173282.0,-19.87,30.0,X-MC,7.29,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.06,-5.85,33.90,26.07,52260.0,-9580.0,154160.0,-30.68,31.0,X-MC,8.72,55.0,-0.18,1.07,0.00,X40,ATH,APPARELS
56,QUESS,424.00,-2.10,-28.83,100.71,42.84,46592.0,-18742.0,46264.0,-53.17,37.0,X-SC,37.14,83.0,-0.40,0.32,0.24,XY24,NTT,MISC
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.68,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.10,-28.83,100.71,42.84,46592.0,-18742.0,46264.0,-53.17,37.0,X-SC,37.14,83.0,-0.40,0.32,0.24,XY24,NTT,MISC
60,RELAXO,1176.00,-0.84,-46.80,186.31,52.31,143870.0,-67939.0,77221.0,-43.37,31.0,X-SC,3.42,91.0,-0.47,0.54,3.36,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.50,-35.21,108.68,35.20,90601.0,-45305.0,83365.0,9.31,23.0,X-SC,12.00,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.70,2.00,21.64,24.08,28022.0,2540.0,129490.0,-6.29,42.0,X-SC,4.52,89.0,0.09,0.90,22.01,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.33,-2.97,25.30,21.58,34403.0,-4158.0,135981.0,-51.07,37.0,X-MC,7.19,56.0,-0.12,0.95,20.39,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.04,-14.52,39.55,19.28,116955.0,-50241.0,295715.0,-25.57,63.0,X-LC,7.78,1.0,-0.43,2.06,9.32,X40,ATH,IT
41,INFY,2275.00,-0.21,4.38,48.37,54.87,157973.0,13705.0,326593.0,-18.88,58.0,X-LC,3.33,2.0,0.09,2.27,13.03,X40,BTT,IT
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.68,XY24,NTT,AUTO
82,VBL,671.64,-0.90,-9.62,50.10,35.67,143022.0,-30369.0,285473.0,-20.30,41.0,X-LC,6.13,4.0,-0.21,1.99,2.91,X40N,ATH,FMCG
43,ITC,452.00,0.04,-0.90,11.44,10.44,22690.0,-1800.0,198338.0,-40.74,45.0,X-LC,0.97,5.0,-0.08,1.38,4.61,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.27,-12.41,107.42,81.67,88263.0,-11644.0,82166.0,7388.89,53.0,L-SC,18.41,271.0,-0.13,0.57,59.12,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.30,-11.92,58.43,39.54,52062.0,-12058.0,89101.0,-31.16,71.0,M-SC,6.91,220.0,-0.23,0.62,25.64,AR,ATH,AUTO
51,MASFIN,398.61,-1.18,-3.67,26.70,22.05,25199.0,-3600.0,94380.0,-16.89,55.0,H-SC,7.02,164.0,-0.14,0.66,37.19,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.32,-8.39,9.18,0.02,14680.0,-14649.0,159910.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.09,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-1.28,7.31,12.00,20.19,22448.0,12750.0,187070.0,-15.78,52.0,X-MC,7.68,68.0,0.57,1.30,23.53,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.32,-8.39,9.18,0.02,14680.0,-14649.0,159910.0,143.86,73.0,M-SC,4.25,234.0,-1.00,1.11,39.09,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.00,-0.04,20.99,20.95,52826.0,-97.0,251671.0,-1.05,75.0,X-LC,12.08,31.0,-0.00,1.75,35.58,X40,ATH,PAINTS
0,5PAISA,593.00,-0.88,-32.71,67.77,12.90,99646.0,-71461.0,147035.0,116.35,66.0,H-SC,14.97,161.0,-0.72,1.02,22.26,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,1.52,-37.84,113.78,32.89,54608.0,-29212.0,47994.0,-693.00,63.0,L-MC,5.36,259.0,-0.53,0.33,32.17,XR,NTT,BANKS
68,SONACOMS,806.63,0.30,-11.92,58.43,39.54,52062.0,-12058.0,89101.0,-31.16,71.0,M-SC,6.91,220.0,-0.23,0.62,25.64,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,44.56
2,50,75.36


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.70
MC    29.24
LC    25.04
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.92
X40      21.07
X40N     11.87
XR        9.60
XY25      9.11
AR        9.00
OX40N     7.74
X200      1.87
SR        1.00
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.11
X-MC    22.84
X-LC    19.90
M-SC    12.03
X-SC     8.15
H-MC     4.70
H-LC     2.98
L-SC     1.41
M-MC     1.37
M-LC     1.18
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.70,-4.93,38.86
IT,13.78,-14.25,73.23
FINANCE,9.47,-14.00,62.17
MISC,7.48,-21.02,70.60
PAINTS,6.01,-6.81,22.68
ELECTRICAL,5.87,-11.31,52.08
INSURANCE,4.43,0.75,34.23
PHARMA,3.85,-1.24,33.64
AUTO,3.35,-19.42,70.63


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3026196.0,21
XR,1283155.0,13
AR,1280177.0,10
X40,1012101.0,14
X40N,746040.0,9
OX40N,679214.0,10
XY25,333711.0,6
SR,277090.0,2
MH,69541.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3518938.0,25
M-SC,1330614.0,15
X-MC,1204979.0,16
X-LC,903489.0,11
X-SC,748576.0,8
H-MC,403696.0,3
L-SC,259204.0,3
H-LC,129657.0,2
M-LC,122138.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1208817.0      6
           AR         867429.0      5
           XR         785758.0      7
M-SC       XY24       778418.0      6
X-MC       X40        439763.0      7
X-LC       X40        395669.0      5
X-MC       XY24       345992.0      3
H-SC       OX40N      310303.0      4
X-SC       X40N       290857.0      3
M-SC       OX40N      281587.0      5
X-SC       XY24       281050.0      3
H-SC       SR         277090.0      2
X-MC       X40N       245369.0      4
X-LC       XY24       225258.0      2
H-MC       AR         217035.0      2
X-LC       X40N       209814.0      2
H-MC       XY24       186661.0      1
X-SC       X40        176669.0      2
X-MC       XY25       173855.0      2
L-SC       XR         171880.0      2
M-SC       XR         148771.0      2
M-LC       XR         122138.0      1
M-SC       AR         121838.0      2
L-SC       OX40N       87324.0      1
H-LC       AR          73875.0      1
X-LC       XY25        72748.0      2
H-SC       MH          69541.0      1
H-LC       X200        55782.0      1
L-MC       XR          54608.0      1
M-MC       XY25        48019.0      1
L-LC       XY25        39089.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
